<a href="https://colab.research.google.com/github/Priya-gawhane/LLM-projects/blob/main/llm1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers torch accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.7 MB/s eta 0:00:00


In [2]:
import torch

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import re
from typing import List, Dict, Any
import warnings
warnings.filterwarnings("ignore")

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Using device: cpu


In [5]:
class ClaimsGenerator:
    def __init__(self, model_name: str = "google/flan-t5-small"):
        """
        Initialize the claims generator with a Hugging Face model.
        Optimized for Google Colab with memory-efficient defaults.

        Args:
            model_name: Name of the Hugging Face model to use
                       Colab-friendly options: "google/flan-t5-small", "google/flan-t5-base",
                       "facebook/bart-large-cnn" (requires GPU)
        """
        self.model_name = model_name
        self.tokenizer = None
        self.model = None
        self.pipeline = None
        self.device = device
        print(f"Initializing with model: {model_name}")
        self.load_model()

    def load_model(self):
        """Load the model and tokenizer with Colab optimizations"""
        try:
            print(f"Loading model: {self.model_name}")

            # Use torch_dtype=torch.float16 for GPU to save memory
            model_kwargs = {"torch_dtype": torch.float16} if self.device.type == "cuda" else {}

            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)

            if "t5" in self.model_name.lower():
                self.model = AutoModelForSeq2SeqLM.from_pretrained(
                    self.model_name,
                    **model_kwargs
                ).to(self.device)

                self.pipeline = pipeline(
                    "text2text-generation",
                    model=self.model,
                    tokenizer=self.tokenizer,
                    device=0 if self.device.type == "cuda" else -1,
                    max_length=256,  # Reduced for Colab
                    do_sample=True,
                    temperature=0.7,
                    return_full_text=False
                )
            else:
                # For BART and other models
                self.pipeline = pipeline(
                    "summarization",
                    model=self.model_name,
                    device=0 if self.device.type == "cuda" else -1,
                    torch_dtype=torch.float16 if self.device.type == "cuda" else torch.float32,
                    max_length=130,  # Reduced for Colab
                    min_length=25,
                    do_sample=True,
                    temperature=0.7
                )

            print("✅ Model loaded successfully!")

        except Exception as e:
            print(f"❌ Error loading model: {e}")
            print("🔄 Trying fallback model...")
            # Fallback to the smallest working model
            self.model_name = "google/flan-t5-small"
            try:
                self.pipeline = pipeline(
                    "text2text-generation",
                    model=self.model_name,
                    device=0 if self.device.type == "cuda" else -1,
                    max_length=200
                )
                print(f"✅ Fallback model loaded: {self.model_name}")
            except Exception as e2:
                print(f"❌ Fallback failed: {e2}")
                raise

    def extract_claims_with_prompting(self, input_text: str) -> List[str]:
        """
        Extract claims using prompt-based generation with T5/FLAN-T5
        """
        prompt = f"""Extract specific factual claims from the following text.
        Present each claim as a separate sentence that can stand alone as a fact.

        Text: {input_text}

        Claims:"""

        try:
            if "t5" in self.model_name.lower():
                response = self.pipeline(prompt,
                                       max_length=300,
                                       num_return_sequences=1,
                                       do_sample=True,
                                       temperature=0.7)
                claims_text = response[0]['generated_text']
            else:
                # For other models, use summarization approach
                response = self.pipeline(input_text,
                                       max_length=200,
                                       min_length=50,
                                       num_return_sequences=1)
                claims_text = response[0]['summary_text']

            # Split into individual claims
            claims = self._split_into_claims(claims_text)
            return claims

        except Exception as e:
            print(f"Error in claim extraction: {e}")
            return self._fallback_claim_extraction(input_text)

    def generate_claims_structured(self, input_text: str) -> Dict[str, List[str]]:
        """
        Generate structured claims categorized by type
        """
        categories = {
            "factual": f"Extract factual statements from: {input_text}",
            "numerical": f"Extract numerical facts and statistics from: {input_text}",
            "temporal": f"Extract time-related information from: {input_text}",
            "causal": f"Extract cause-and-effect relationships from: {input_text}"
        }

        structured_claims = {}

        for category, prompt in categories.items():
            try:
                if "t5" in self.model_name.lower():
                    response = self.pipeline(prompt,
                                           max_length=200,
                                           num_return_sequences=1,
                                           temperature=0.6)
                    claims_text = response[0]['generated_text']
                    structured_claims[category] = self._split_into_claims(claims_text)
                else:
                    # For non-T5 models, use the original text with category focus
                    structured_claims[category] = self._extract_category_claims(input_text, category)

            except Exception as e:
                print(f"Error extracting {category} claims: {e}")
                structured_claims[category] = []

        return structured_claims

    def _split_into_claims(self, text: str) -> List[str]:
        """Split generated text into individual claims"""
        # Clean and split the text
        text = text.strip()

        # Split by sentences and common claim separators
        claims = re.split(r'[.!?]\s+|\n\s*[-•]\s*|\n\d+\.\s*', text)

        # Clean and filter claims
        cleaned_claims = []
        for claim in claims:
            claim = claim.strip().strip('.,!?-•')
            if len(claim) > 10 and not claim.startswith(('Extract', 'Claims:', 'Text:')):
                cleaned_claims.append(claim)

        return cleaned_claims[:5]  # Return top 5 claims

    def _extract_category_claims(self, text: str, category: str) -> List[str]:
        """Extract claims for specific categories using rule-based approach"""
        sentences = re.split(r'[.!?]+', text)
        category_claims = []

        if category == "numerical":
            for sentence in sentences:
                if re.search(r'\d+', sentence):
                    category_claims.append(sentence.strip())
        elif category == "temporal":
            for sentence in sentences:
                if re.search(r'\b(year|month|day|century|ago|since|when|during|after|before)\b', sentence, re.IGNORECASE):
                    category_claims.append(sentence.strip())
        elif category == "causal":
            for sentence in sentences:
                if re.search(r'\b(because|due to|caused by|results in|leads to|therefore)\b', sentence, re.IGNORECASE):
                    category_claims.append(sentence.strip())
        else:  # factual
            category_claims = sentences[:3]  # First few sentences as factual claims

        return [claim for claim in category_claims if len(claim) > 10][:3]

    def _fallback_claim_extraction(self, text: str) -> List[str]:
        """Simple fallback method for claim extraction"""
        sentences = re.split(r'[.!?]+', text)
        claims = [s.strip() for s in sentences if len(s.strip()) > 15]
        return claims[:5]

    def process_input(self, input_text: str, method: str = "prompting") -> Dict[str, Any]:
        """
        Main method to process input and generate claims

        Args:
            input_text: The input text to extract claims from
            method: "prompting" or "structured"

        Returns:
            Dictionary containing claims and metadata
        """
        if not input_text or len(input_text.strip()) < 10:
            return {"error": "Input text is too short or empty"}

        result = {
            "input": input_text,
            "model_used": self.model_name,
            "method": method,
            "claims": []
        }

        try:
            if method == "structured":
                result["claims"] = self.generate_claims_structured(input_text)
            else:
                result["claims"] = self.extract_claims_with_prompting(input_text)

            result["total_claims"] = len(result["claims"]) if isinstance(result["claims"], list) else sum(len(v) for v in result["claims"].values())

        except Exception as e:
            result["error"] = f"Processing failed: {e}"
            result["claims"] = self._fallback_claim_extraction(input_text)

        return result

In [6]:
def main():
    print("🚀 Starting Claims Generator Demo for Google Colab")
    print("="*60)

    # Sample input texts for testing
    sample_inputs = [
        """
        Climate change is causing global temperatures to rise at an unprecedented rate.
        The last decade has been the warmest on record, with average temperatures
        increasing by 1.2 degrees Celsius since pre-industrial times. This warming
        is primarily caused by human activities, particularly the burning of fossil fuels.
        As a result, polar ice caps are melting, leading to rising sea levels that
        threaten coastal communities worldwide.
        """,

        """
        The company reported a 25% increase in revenue for Q3 2023, reaching $2.5 billion.
        This growth was driven by strong performance in the AI division, which launched
        three new products this year. The CEO announced plans to hire 500 new employees
        by the end of 2024 to support expansion into European markets.
        """
    ]

    # Initialize the claims generator with Colab-friendly model
    print("🤖 Initializing Claims Generator...")
    try:
        generator = ClaimsGenerator("google/flan-t5-small")  # Best for Colab free tier
    except Exception as e:
        print(f"❌ Initialization failed: {e}")
        return

    # Process each sample input
    for i, input_text in enumerate(sample_inputs, 1):
        print(f"\n{'='*60}")
        print(f"📝 PROCESSING SAMPLE {i}")
        print(f"{'='*60}")
        print(f"Input preview: {input_text[:100]}...")

        # Method 1: Simple prompting
        print(f"\n🔍 Method 1: Prompting-based extraction")
        print("-" * 40)
        try:
            result1 = generator.process_input(input_text, method="prompting")

            if "error" not in result1:
                print(f"✅ Extracted {len(result1['claims'])} claims:")
                for j, claim in enumerate(result1["claims"], 1):
                    print(f"   {j}. {claim}")
            else:
                print(f"❌ Error: {result1['error']}")
        except Exception as e:
            print(f"❌ Processing error: {e}")

        # Method 2: Structured extraction
        print(f"\n📊 Method 2: Structured extraction")
        print("-" * 40)
        try:
            result2 = generator.process_input(input_text, method="structured")

            if "error" not in result2 and isinstance(result2["claims"], dict):
                total_claims = sum(len(claims) for claims in result2["claims"].values())
                print(f"✅ Extracted {total_claims} categorized claims:")

                for category, claims in result2["claims"].items():
                    if claims:
                        print(f"\n   📋 {category.upper()} CLAIMS:")
                        for j, claim in enumerate(claims, 1):
                            print(f"      {j}. {claim}")
            else:
                print(f"❌ Error: {result2.get('error', 'Unknown error')}")
        except Exception as e:
            print(f"❌ Processing error: {e}")

In [7]:
def cleanup_model(generator):
    """Clean up model from memory"""
    if hasattr(generator, 'model') and generator.model is not None:
        del generator.model
    if hasattr(generator, 'pipeline') and generator.pipeline is not None:
        del generator.pipeline
    torch.cuda.empty_cache() if torch.cuda.is_available() else None
    print("🧹 Model cleaned from memory")

if __name__ == "__main__":
    main()

🚀 Starting Claims Generator Demo for Google Colab
🤖 Initializing Claims Generator...
Initializing with model: google/flan-t5-small
Loading model: google/flan-t5-small


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu


✅ Model loaded successfully!

📝 PROCESSING SAMPLE 1
Input preview: 
        Climate change is causing global temperatures to rise at an unprecedented rate. 
        Th...

🔍 Method 1: Prompting-based extraction
----------------------------------------
Error in claim extraction: The following `model_kwargs` are not used by the model: ['return_full_text'] (note: typos in the generate arguments will also show up in this list)
✅ Extracted 5 claims:
   1. Climate change is causing global temperatures to rise at an unprecedented rate
   2. The last decade has been the warmest on record, with average temperatures 
        increasing by 1
   3. 2 degrees Celsius since pre-industrial times
   4. This warming 
        is primarily caused by human activities, particularly the burning of fossil fuels
   5. As a result, polar ice caps are melting, leading to rising sea levels that 
        threaten coastal communities worldwide

📊 Method 2: Structured extraction
------------------------------------